# Jake AI Fine-tuning

This notebook will fine-tune Llama 2 to generate Jake-style stories using LoRA.

In [ ]:
# Install additional required packages
!pip install -q bitsandbytes
!pip install -q git+https://github.com/huggingface/peft.git
!pip install -q git+https://github.com/huggingface/transformers.git

In [ ]:
# Set up Hugging Face access token
from huggingface_hub import login
login()  # You'll need to enter your HF token here

In [ ]:
# Mount Google Drive to access training data
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import json
import torch
from datasets import Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling
)
from peft import get_peft_model, LoraConfig, TaskType

# Load and prepare the dataset
def prepare_dataset(json_file):
    with open(json_file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    formatted_data = []
    for conv in data['conversations']:
        for msg in conv['messages']:
            if msg['role'] == 'assistant':
                formatted_data.append({
                    'text': f"<s>[INST] Tell me a Jake story [/INST] {msg['content']}</s>"
                })
    
    return Dataset.from_list(formatted_data)

# Create PEFT config
def create_peft_config():
    return LoraConfig(
        task_type=TaskType.CAUSAL_LM,
        r=8,
        lora_alpha=32,
        lora_dropout=0.1,
        target_modules=["q_proj", "v_proj"]
    )

In [ ]:
# Load model and tokenizer
model_name = "meta-llama/Llama-2-7b-hf"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)

In [ ]:
# Prepare dataset
# Update this path to where you uploaded the JSON file
dataset = prepare_dataset("/content/drive/MyDrive/jake_training.json")

# Create PEFT config and wrap model
peft_config = create_peft_config()
model = get_peft_model(model, peft_config)

In [ ]:
# Set up training arguments
training_args = TrainingArguments(
    output_dir="jake-ai-model",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    fp16=True,
    save_steps=50,
    logging_steps=10,
)

# Create trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

In [ ]:
# Start training
trainer.train()

In [ ]:
# Save the model
trainer.save_model("jake-ai-final")

# Test the model
from transformers import pipeline

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=1000,
    temperature=0.7,
    top_p=0.9,
    repetition_penalty=1.1
)

# Test generation
prompt = "[INST] Tell me about Jake's latest sales adventure [/INST]"
print(pipe(prompt)[0]['generated_text'])